In [3]:
import json
import jsonlines
import networkx as nx
import hypernetx as hnx
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint
from refined.inference.processor import Refined


 No module named 'celluloid'. If you need to use hypernetx.algorithms.contagion, please install additional packages by running the following command: pip install .['all']


/Users/samytlee/opt/anaconda3/envs/event_hgraph_preprocess/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
refined = Refined.from_pretrained(model_name='wikipedia_model_with_numbers',
                                  entity_set="wikipedia")

In [4]:
def save_json(data, filepath=r'new_data.json'):
    with open(filepath, 'w') as fp:
        json.dump(data, fp, indent=4)

In [5]:
def merge_sentences(sentences):
    sentence_list = [" ".join(sentence_word_list) for sentence_word_list in sentences] # merge the words into sentences
    paragraph = " ".join(sentence_list)
    return paragraph

def prepare_events(datum):
    # words_flattened = [word for sentence in datum['sentences'] for word in sentence]
    for event in datum['events']:
        # trigger = event['trigger']
        arguments = event['arguments']
        arguments_obj = [
            { 
                'argument_id': argument, 
                'argument_word': argument,
            }
            for argument in arguments
            ]
        event['arguments'] = arguments_obj
    return datum['events']

def link_entities(events, paragraph):
    spans = refined.process_text(paragraph)
    for span in spans:
        entity_word = span.text
        for event in events:
            for argument in event['arguments']:
                if argument['argument_word'] == entity_word:
                    if span.predicted_entity != None and span.predicted_entity.wikidata_entity_id != None:
                        entity_id = span.predicted_entity.wikidata_entity_id
                        entity_title = span.predicted_entity.wikipedia_entity_title
                        argument['argument_id'] = entity_id
                        argument['entity_title'] = entity_title
                    argument['entity_type'] = span.coarse_mention_type
    return events

def transform_dataset(dataset):
    transformed_dataset = {}
    for index, datum in enumerate(dataset):
        print("{}/{}".format(index, len(dataset)))
        paragraph = merge_sentences(datum['content'])
        events = prepare_events(datum)
        events = link_entities(events, paragraph)
        if events == []: continue
        doc_key = datum['doc_id']
        source_url = datum['url']
        if doc_key not in transformed_dataset.keys():
            transformed_dataset[doc_key] = {
                "doc_id": doc_key,
                "source_url": source_url,
                "content": datum['content'],
                "summary": datum['summary'],
                "events": []
            }
        transformed_dataset[doc_key]['events'] += events
    return list(transformed_dataset.values())

def remove_duplicates(dataset):
    reverse_index_url = defaultdict(list)
    kept_dataset = []
    for index, datum in enumerate(dataset):
        if datum['source_url'] in reverse_index_url.keys():
            previous_data = reverse_index_url[datum['source_url']]
            duplicate = False
            for previous_datum in previous_data:
                if " ".join(datum['content'][0][0]) == " ".join(previous_datum['content'][0][0]):
                    duplicate = True
                    break
            if duplicate: continue
        reverse_index_url[datum['source_url']].append(datum)
        kept_dataset.append(datum)
    return kept_dataset


In [ ]:
# dataset = [datum for datum in dev_reader] + [datum for datum in test_reader] + [datum for datum in train_reader]
# dataset = [datum for datum in dev_reader]
dataset = json.load(open('data/result/RAMS/gpt_events_dev.json'))
transformed_dataset = transform_dataset(dataset)
transformed_dataset = remove_duplicates(dataset)
# second round linking
for datum in dataset:
    datum['events'] = second_round_linking(datum['events'])
save_json(transformed_dataset, r'data/result/RAMS/gpt_events_dev_linked.json')

In [ ]:
def second_round_linking(events):
    for event in events:
        for argument in event['arguments']:
            if argument['argument_id'] == argument['argument_word']:
                if 'entity_type' in argument.keys(): del argument['entity_type']
                spans = refined.process_text(argument['argument_word'])
                for span in spans:
                    if span.predicted_entity != None and span.predicted_entity.wikidata_entity_id != None:
                        entity_id = span.predicted_entity.wikidata_entity_id
                        entity_title = span.predicted_entity.wikipedia_entity_title
                        argument['argument_id'] = entity_id
                        argument['entity_title'] = entity_title
                        argument['entity_type'] = span.coarse_mention_type
    return events

In [ ]:
dataset = json.load(open('data/result/RAMS/gpt_events_dev_linked.json'))
for datum in dataset:
    datum['events'] = second_round_linking(datum['events'])
save_json(dataset, r'data/result/RAMS/gpt_events_dev_linked.json')

In [7]:
from collections import defaultdict
# create node link graph
def construct_network(docs):
    nodes_dict = {}
    event_hyperedges_dict = {}
    links = []
    sub_event_links = defaultdict(list)
    sub_event_hyperedges_dict = defaultdict(list)
    for doc in docs:
        doc_id = doc['doc_id']
        doc_url = doc['source_url']
        event_triggers = []
        event_arguments = set()
        event_triggers = sorted(list(set([sub_event['trigger'] for sub_event in doc['events']])))
        # check gpt response for error
        if len(event_triggers) == 1:
            trigger = event_triggers[0]
            if trigger.startswith("I'm sorry"): continue
            if trigger.startswith("No event"): continue
            if trigger.startswith("There are no"): continue
            if trigger.startswith("There is no"): continue
            if trigger.startswith("I'm unable to "): continue

        event_hyperedge_id = str(doc_id) +  "-" + "-".join(event_triggers)
        for sub_event in doc['events']:
            arguments = sub_event['arguments']
            # create an entity node for each argument
            for argument in arguments:
                if argument['argument_id'] == argument['argument_word']: 
                    argument['argument_id'] = argument['argument_id'] + '-' + str(doc_id)
                argument_id = argument['argument_id']
                argument_word = argument['argument_word']
                argument_title = argument['entity_title'] if 'entity_title' in argument else argument_word
                argument_entity_type = argument['entity_type'] if 'entity_type' in argument else "None"
                # argument_span = argument['argument_span']
                # argument_role = argument['argument_role']
                if argument_id not in nodes_dict.keys():
                    nodes_dict[argument_id] = {
                        "id": argument_id, 
                        "title": argument_title,
                        "entity_type": argument_entity_type,
                        "type": "entity",
                        # "argument_role": argument_role,
                        "mentions": [
                            {
                                "doc_id": doc_id,
                                "mention": argument_word,
                                # "span": {'start': argument_span[0], 'end': argument_span[1]}
                            }
                        ]
                    }
                else:
                    nodes_dict[argument_id]['mentions'].append(
                        {
                            "doc_id": doc_id,
                            "mention": argument_word,
                            # "span": {'start': argument_span[0], 'end': argument_span[1]}
                        }
                    )
            argument_ids = [argument['argument_id'] for argument in arguments]
            sorted_argument_ids = sorted(argument_ids)
            # create hyperedge 
            trigger_id = sub_event['trigger'] 
            sub_event_hyper_edge_id = trigger_id  + "-" + str(doc_id)
            sub_event_hyperedges_dict[event_hyperedge_id].append({
                'id': sub_event_hyper_edge_id,
                'type': "subevent_hyper_edge",
                "trigger": trigger_id,
                # "trigger_type": trigger_type,
                "arguments": sorted_argument_ids,
                "doc_id": doc_id,
            })
            for argument_id in argument_ids:
                sub_event_links[event_hyperedge_id].append((sub_event_hyper_edge_id, argument_id))
            event_arguments.update(argument_ids)

        event_hyperedges_dict[event_hyperedge_id] = {
            'id': event_hyperedge_id,
            'type': "hyper_edge",
            "trigger": event_hyperedge_id,
            "arguments": list(event_arguments),
            "doc_id": doc_id,
            "summary": doc['summary'],
            "content": doc['content']
        }
        for argument_id in list(event_arguments):
            links.append((event_hyperedge_id, argument_id))

    return nodes_dict, event_hyperedges_dict, links, sub_event_hyperedges_dict, sub_event_links

def merge_RAMS(dataset):
    nodes_dict, hyper_edges_dict, links, sub_event_hyperedges, sub_event_links_dict = construct_network(dataset)
    B = nx.Graph()
    B.add_nodes_from(list(hyper_edges_dict.keys()), bipartite=0)
    B.add_nodes_from(list(nodes_dict.keys()), bipartite=1)
    B.add_edges_from(links)

    return B, nodes_dict, hyper_edges_dict, links, sub_event_hyperedges, sub_event_links_dict


In [8]:
transformed_dataset = json.load(open('data/result/RAMS/gpt_events_dev_linked.json'))
B, nodes_dict, hyper_edges_dict, links, sub_event_hyperedges, sub_event_links_dict = merge_RAMS(transformed_dataset)



In [9]:
H = hnx.Hypergraph.from_bipartite(B)
list(H.shape)

[3024, 617]

In [10]:
event_hgraph_data = nx.node_link_data(B)
save_json(event_hgraph_data, r'data/result/RAMS/gpt_biHgraph_dev/hgraph.json')
save_json(nodes_dict, r'data/result/RAMS/gpt_biHgraph_dev/nodes.json')
save_json(hyper_edges_dict, r'data/result/RAMS/gpt_biHgraph_dev/hyperedges.json')

save_json(sub_event_hyperedges, r'data/result/RAMS/gpt_biHgraph_dev/sub_event_hyperedges.json')
save_json(sub_event_links_dict, r'data/result/RAMS/gpt_biHgraph_dev/sub_event_links.json')

In [11]:
def transform_frontend(nodes, links, nodes_dict, hyper_edges_dict):
    res_nodes = []
    res_links = []
    for node in nodes:
        if node in nodes_dict:
            res_nodes.append(nodes_dict[node])
        else:
            res_nodes.append(hyper_edges_dict[node])
    for link in links:
        source = link[0]
        target = link[1]
        res_links.append({
            "source": source,
            "target": target,
        })
    print(len(res_nodes))
    return {
        "nodes": res_nodes, 
        "links": res_links
    }

In [12]:
BH = H.bipartite()
network = transform_frontend(list(BH.nodes), list(BH.edges), nodes_dict, hyper_edges_dict)
save_json(network, 'data/result/RAMS/gpt_dev_hgraph.json')

3641
